In [ ]:
import pandas as pd

df_output = pd.read_csv('input_files/PMS_repl_translations_sg_20221018.csv')

df_output["Modified"] = ""
col_modified = df_output['Modified']
df_output.drop(labels=['Modified'], axis=1,inplace = True)
df_output.insert(0, 'Modified', col_modified)
df_output["DE (Translation Tool)"]=df_output["DE (Translation Tool)"].values.astype('object')
df_output["FR (Translation Tool)"]=df_output["FR (Translation Tool)"].values.astype('object')
df_output["IT (Translation Tool)"]=df_output["IT (Translation Tool)"].values.astype('object')

df_master = pd.read_csv('input_files/PMS_repl_translations_sg_20221018_dictionary.csv')

df_output.dtypes

In [ ]:
from rapidfuzz import fuzz, process
import deepl

auth_key = '325c848e-727d-6daf-ec26-cee3d14b11cc:fx'
translator = deepl.Translator(auth_key)

def deepl_translate(word):
    translation_DE = translator.translate_text(word, target_lang="DE", formality="more").text
    translation_FR = translator.translate_text(word, target_lang="FR", formality="more").text
    translation_IT = translator.translate_text(word, target_lang="IT", formality="more").text
    return translation_DE,translation_FR,translation_IT,'Deepl'

def fuzzy_search(word):
    df_master_filled = df_master['EN'].fillna('')
    most_similar = process.extractOne(word, df_master_filled, scorer=fuzz.token_sort_ratio, score_cutoff = 90)
    if most_similar:
        df_temp = df_master.loc[df_master['EN'] == most_similar[0]]
        return df_temp['DE'].values[0], df_temp['FR'].values[0], df_temp['IT'].values[0], 'Fuzzy: '+most_similar[0]+',  %8.2f' % most_similar[1]
    else:
        return deepl_translate(word)

def find_EN_translations(word):
    df_temp = df_master.loc[df_master['EN'] == word]
    if df_temp.empty:
        return fuzzy_search(word)
    else:
        de_temp = ''
        fr_temp = '' 
        it_temp = ''
        for i in df_temp:
            if (df_temp['DE'].values[0]):
                de_temp = df_temp['DE'].values[0]
            if (df_temp['FR'].values[0]):
                fr_temp = df_temp['FR'].values[0]
            if (df_temp['IT'].values[0]):
                it_temp = df_temp['IT'].values[0]
        print(de_temp+fr_temp+it_temp)
        return de_temp, fr_temp, it_temp, 'Identical match'


In [ ]:
df_output_temp = df_output[df_output['EN'].notna()]
for i in df_output_temp.index:
    # Find row with translations in masterfile
    cell_DE, cell_FR, cell_IT, cell_mod = find_EN_translations(df_output_temp.at[i, 'EN'])
    df_output.at[i, 'DE (Translation Tool)'] = cell_DE
    df_output.at[i, 'FR (Translation Tool)'] = cell_FR
    df_output.at[i, 'IT (Translation Tool)'] = cell_IT
    df_output.at[i, 'Modified'] = cell_mod

In [ ]:
df_output.to_csv("output_files/PMS_repl_translations_sg_20221018_updated.csv",index=None)